# This is the file we use to combine and label the songs based on whether a song had ever made it to the top 30 of the chart
## Although we did not use this criteria eventually, the data we use now is built upon output of this file as this combines all the unique songs across songs collected for all 3 years and keep the 

In [1]:
import datetime
import os
from os import system
from datetime import timedelta, date
import pandas as pd
import glob

In [2]:
def assign_label(track_name, hit_list, non_hit_list):
    if track_name in hit_list:
        return 1
    elif track_name in non_hit_list:
        return 0
    else:
        return -1 #indicate something went wrong
def update_streams(track_name, highest_stream):
    return highest_stream[track_name]

In [3]:
def label_data(hit_song_criteria):
    all_filenames = [i for i in glob.glob('feature_*')]
    data = pd.concat([pd.read_csv(f) for f in all_filenames ]) #combine all csv
    
    hit_list = []        # list of hit songs
    non_hit_list = []    # list of non hit songs 
    highest_stream = {}  # highest stream value we can obtain form the data
    
    
    for index, row in data.iterrows():
        if row['Track Name'] not in highest_stream or int(row['Streams']) > int(highest_stream[row['Track Name']]):
            highest_stream[row['Track Name']] = row['Streams']
        if int(row['Position']) <= hit_song_criteria and row['Track Name'] not in hit_list:
            hit_list.append(row['Track Name'])
    for index, row in data.iterrows():
        if int(row['Position']) > hit_song_criteria and row['Track Name'] not in hit_list and row['Track Name'] not in non_hit_list:
            non_hit_list.append(row['Track Name'])
    data = data.drop_duplicates(['Track Name'])        
    data['Label'] = data['Track Name'].apply(lambda track_name: assign_label(track_name, hit_list, non_hit_list))
    data['Streams'] = data['Track Name'].apply(lambda track_name: update_streams(track_name, highest_stream))
    del data["Position"]
    data = data.sort_values(by = 'Label', ascending=False)
    data.to_csv('labeled_spotify_data.csv',index = False)
    print("Label finished, the labeled file is saved as labeled_spotify_data.csv")
    print("number of hit song: ", len(hit_list))
    print("number of non hit song: ", len(non_hit_list))

In [4]:
# songs that managed to hit the top 10 list will be considered as "hit song"
hit_song_criteria = 30
print("Songs that managed to once hit top {} in the list will be considered as hit song".format(hit_song_criteria))
label_data(hit_song_criteria)

Songs that managed to once hit top 30 in the list will be considered as hit song
Label finished, the labeled file is saved as labeled_spotify_data.csv
number of hit song:  1035
number of non hit song:  3745


In [5]:
def remove_redundant_columns(csv_file):
    data = pd.read_csv(csv_file)
    col_to_del = ["URL", "trackID", "type", "id", "uri", "track_href"]
    print("removing columns ","URL,", "trackID,", "type,", "id,", "uri,", "track_href,")
    for col in col_to_del:
        try:
            del data[col]
        except Exceptions as e:
            print(e)
    data.to_csv('labeled_spotify_data_clean.csv',index = False)
    print("only essential features kept in labeled_spotify_data_clean.csv")
    
remove_redundant_columns('labeled_spotify_data.csv')

removing columns  URL, trackID, type, id, uri, track_href,
only essential features kept in labeled_spotify_data_clean.csv
